In [19]:
import numpy as np
import pandas as pd

In [20]:
X = pd.read_csv('Hotel Reservations.csv') # full model
X.drop(['booking_status', 'Booking_ID'], axis = 1, inplace = True)

y = pd.read_csv("Hotel Reservations.csv")['booking_status']

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

X = pd.get_dummies(X)
le = LabelEncoder()
y = pd.DataFrame(le.fit_transform(y))
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1, test_size=None, train_size=0.3)

## Kfold CV

In [41]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5,shuffle=True,random_state=8)

for i, (train_index, test_index) in enumerate(kf.split(X)):
    print(X.iloc[train_index,:])
    break


       no_of_adults  no_of_children  no_of_weekend_nights  no_of_week_nights  \
0                 2               0                     1                  2   
1                 2               0                     2                  3   
2                 1               0                     2                  1   
3                 2               0                     0                  2   
4                 2               0                     1                  1   
...             ...             ...                   ...                ...   
36268             1               0                     0                  3   
36269             2               2                     0                  1   
36271             2               0                     1                  3   
36272             2               0                     2                  6   
36274             2               0                     1                  2   

       required_car_parking_space  lead

In [40]:
X.shape

(36275, 30)

In [18]:
X_train.loc[10877,:]

no_of_adults                               1.0
no_of_children                             0.0
no_of_weekend_nights                       0.0
no_of_week_nights                          2.0
required_car_parking_space                 0.0
lead_time                                  1.0
arrival_year                            2017.0
arrival_month                              9.0
arrival_date                              29.0
repeated_guest                             0.0
no_of_previous_cancellations               0.0
no_of_previous_bookings_not_canceled       0.0
avg_price_per_room                        85.0
no_of_special_requests                     0.0
type_of_meal_plan_Meal Plan 1              1.0
type_of_meal_plan_Meal Plan 2              0.0
type_of_meal_plan_Meal Plan 3              0.0
type_of_meal_plan_Not Selected             0.0
room_type_reserved_Room_Type 1             1.0
room_type_reserved_Room_Type 2             0.0
room_type_reserved_Room_Type 3             0.0
room_type_res

In [38]:
X_train.loc[train_index,:]


KeyError: '[0, 2, 3, 4, 6, 11, 17, 19, 22, 23, 24, 29, 30, 31, 32, 35, 36, 37, 38, 44, 45, 50, 55, 56, 58, 60, 63, 66, 67, 72, 74, 75, 77, 79, 80, 82, 84, 85, 88, 89, 90, 91, 92, 95, 98, 99, 102, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 117, 118, 119, 121, 125, 127, 128, 129, 130, 131, 132, 133, 135, 138, 140, 142, 144, 146, 147, 149, 153, 155, 156, 159, 160, 163, 164, 168, 170, 171, 172, 175, 178, 179, 181, 182, 185, 187, 188, 191, 193, 195, 196, 199, 201, 202, 203, 204, 205, 207, 209, 211, 212, 216, 218, 219, 220, 221, 222, 223, 224, 227, 230, 231, 232, 233, 234, 236, 237, 239, 244, 245, 246, 249, 250, 252, 253, 254, 255, 256, 259, 262, 265, 266, 267, 268, 271, 272, 273, 274, 275, 278, 282, 283, 285, 286, 288, 292, 296, 297, 298, 299, 300, 301, 302, 303, 306, 307, 308, 309, 310, 312, 313, 314, 316, 317, 319, 320, 321, 322, 324, 325, 328, 330, 331, 332, 335, 340, 342, 343, 345, 346, 348, 351, 352, 353, 354, 356, 357, 358, 359, 361, 362, 363, 364, 365, 368, 369, 370, 371, 372,

In [46]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

# create K-fold object with 5 folds
kf = RepeatedKFold(n_splits=5, random_state=8)
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(),
    "XGBoost": xgb.XGBClassifier(max_depth=10, learning_rate=0.1)
}
cv_scores = {}
for name, model in models.items():
    pipeline = Pipeline(steps=[("scaler", StandardScaler()), ("clf", model)])
    for train_index, test_index in kf.split(X_train):
        pipeline.fit(X_train.iloc[train_index, :], y_train.iloc[train_index])
        p_pred = pipeline.predict_proba(X_train.iloc[test_index, :])[:, 1]
        cv_scores[name] = roc_auc_score(y_train.iloc[test_index], p_pred)
print(cv_scores)


/opt/homebrew/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/homebrew/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/homebrew/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/homebrew/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

{'Logistic Regression': 0.8545072661217076, 'Random Forest': 0.9229514001139179, 'XGBoost': 0.9366889114672331}


In [53]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifierCV
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB

from sklearn.neural_network import MLPClassifier

# create K-fold object with 5 folds
models = {
    "Naive Bayes" : GaussianNB(),
    "Logistic Regression": LogisticRegression(),
    "RidgeClassifierCV": RidgeClassifierCV(alphas=[1e-3, 1e-2, 1e-1, 1]),
    "Neural_Network": MLPClassifier(hidden_layer_sizes = 100,learning_rate_init = 0.01)
    "Random Forest": RandomForestClassifier(),
    "XGBoost": xgb.XGBClassifier(max_depth=10, learning_rate=0.1),
}
cv_scores = {}
for name, model in models.items():
    pipeline = Pipeline(steps=[("scaler", StandardScaler()), ("clf", model)])
    cur_score = np.mean(cross_val_score(pipeline, X_train, y_train, cv=3, scoring = "roc_auc"))
    cv_scores[name] = cur_score
print(cv_scores)


/opt/homebrew/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/homebrew/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/homebrew/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/homebrew/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

{'Naive Bayes': 0.7863972592626509, 'Logistic Regression': 0.8602632512178481, 'Random Forest': 0.9300348880172562, 'XGBoost': 0.940237988887849, 'RidgeClassifierCV': 0.8583293194090045, 'Neural_Network': 0.8896394415050892}
